<a href="https://colab.research.google.com/github/kengo6122024/HelloGit/blob/main/xgboost_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

# データの読み込み
train_data = pd.read_csv('/content/drive/MyDrive/GCI_compe_1/data/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/GCI_compe_1/data/test.csv')

In [ ]:
train_data.head()

,PassengerId,Perished,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,1,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
from sklearn.preprocessing import StandardScaler

# 正規表現から敬称を抽出する関数 - *1
def extract_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1)
    return ""

#敬称から年齢の欠損値を中央値で補完する関数 - *2
def fill_age(row, median_ages):
    if pd.isnull(row['Age']) and row['Title'] in median_ages:
        return median_ages[row['Title']]
    else:
        return row['Age']

# 1'Fare' 列の標準化
scaler = StandardScaler()
train_data[['Fare']] = scaler.fit_transform(train_data[['Fare']])

# 2. Cabinの列を削除
train_data_cleaned = train_data.drop(columns=['Cabin'])

# 3. 'Sex'と'Embarked' をダミー化
train_data_dummy = pd.get_dummies(train_data_cleaned, columns=['Sex', 'Embarked'])

# 4. *1,*2を利用
train_data_dummy['Title'] = train_data_dummy['Name'].apply(extract_title)
median_ages = train_data_dummy.groupby('Title')['Age'].median()
train_data_dummy['Age'] = train_data_dummy.apply(lambda row: fill_age(row, median_ages), axis=1)

# 5.チケット番号から同室者がいるかどうかを新たな特徴量に追加
ticket_counts = train_data_dummy['Ticket'].value_counts()
train_data_dummy['Has_Companions'] = train_data_dummy['Ticket'].apply(lambda ticket: 1 if ticket_counts[ticket] > 1 else 0)

# 6'Ticket'と'Name'の列はもう不要なので削除
train_data_dummy = train_data_dummy.drop(columns=['Ticket', 'Name'])

#同様の前処理をテストデータにも行う

# 1
test_data[['Fare']] = scaler.fit_transform(test_data[['Fare']])

#2
test_data_cleaned = test_data.drop(columns=['Cabin'])

#3
test_data_dummy = pd.get_dummies(test_data_cleaned, columns=['Sex', 'Embarked'])

#4
test_data_dummy['Title'] = test_data_dummy['Name'].apply(extract_title)
test_data_dummy['Age'] = test_data_dummy.apply(lambda row: fill_age(row, median_ages), axis=1)

#5
ticket_counts = train_data['Ticket'].value_counts()
test_data_dummy['Has_Companions'] = test_data_dummy['Ticket'].apply(lambda ticket: 1 if ticket in ticket_counts and ticket_counts[ticket] > 1 else 0)

#6
test_data_final = test_data_dummy.drop(columns=['Ticket', 'Name', 'Title'])

median_fare = train_data['Fare'].median()
test_data_final['Fare'].fillna(median_fare, inplace=True)

In [ ]:
train_data_dummy.head()

,PassengerId,Perished,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Title,Has_Companions
0,1,1,3,22.0,1,0,-0.502445,0,1,0,0,1,Mr,0
1,2,0,1,38.0,1,0,0.786845,1,0,1,0,0,Mrs,0
2,3,0,3,26.0,0,0,-0.488854,1,0,0,0,1,Miss,0
3,4,0,1,35.0,1,0,0.420730,1,0,0,0,1,Mrs,1
4,5,1,3,35.0,0,0,-0.486337,0,1,0,0,1,Mr,0


In [ ]:
# 説明変数を抽出
X = train_data_dummy.drop(columns=['PassengerId', 'Perished', 'Title','Has_Companions'])

# 目的変数を抽出
y = train_data_dummy['Perished']

X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import xgboost as xgb
import optuna

def objective(trial):
    dtrain = xgb.DMatrix(X_train.values, label=y_train.values)
    dvalid = xgb.DMatrix(X_val.values, label=y_val.values)

    param = {
        'verbosity': 0,
        'objective': 'binary:logistic',
        'booster': trial.suggest_categorical('booster', ['gbtree', 'dart']),
        'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
        'max_depth': trial.suggest_int('max_depth', 1, 9),
        'eta': trial.suggest_loguniform('eta', 1e-8, 1.0),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
        'eval_metric': 'logloss',
        'random_state':42
    }

    bst = xgb.train(param, dtrain, evals=[(dvalid, 'validation')], verbose_eval=False)
    preds = bst.predict(dvalid)
    pred_labels = (preds >= 0.5).astype(int)
    accuracy = (pred_labels == y_val).mean()
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000)

best_trial = study.best_trial
print('Best Score:', best_trial.value)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2023-11-10 19:23:59,095] A new study created in memory with name: no-name-b0df1195-a206-4c13-b515-3ea072f3460e
[I 2023-11-10 19:24:00,739] Trial 0 finished with value: 0.5865921787709497 and parameters: {'booster': 'dart', 'lambda': 0.33525468242796963, 'alpha': 0.003889136363457927, 'max_depth': 1, 'eta': 7.16788023581252e-07, 'gamma': 3.7328306826096547e-06, 'grow_policy': 'lossguide'}. Best is trial 0 with value: 0.5865921787709497.
[I 2023-11-10 19:24:00,767] Trial 1 finished with value: 0.5865921787709497 and parameters: {'booster': 'dart', 'lambda': 0.00048652626372975937, 'alpha': 0.014996786770800893, 'max_depth': 2, 'eta': 7.84364976828144e-08, 'gamma': 1.7871788589348218e-08, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 0.5865921787709497.
[I 2023-11-10 19:24:00,825] Trial 2 finished with value: 0.5865921787709497 and parameters: {'booster': 'gbtree', 'lambda': 7.508161411974067e-08, 'alpha': 2.214281488508048e-07, 'max_depth': 5, 'eta': 2.3000923391258142e-07,

Best Score: 0.8715083798882681
Number of finished trials: 1000
Best trial: {'booster': 'gbtree', 'lambda': 0.34451902315408983, 'alpha': 2.091471774031124e-06, 'max_depth': 8, 'eta': 0.13973018905726448, 'gamma': 0.0015579173952797707, 'grow_policy': 'lossguide'}


In [ ]:
import numpy as np
# パラメータの設定
params = {
    'booster': 'gbtree',
    'lambda': 0.34451902315408983,
    'alpha': 2.091471774031124e-06,
    'max_depth': 8,
    'eta': 0.13973018905726448,
    'gamma': 0.0015579173952797707,
    'grow_policy': 'lossguide'
}

# DMatrix を作成（XGBoost 用のデータ形式）
dtrain = xgb.DMatrix(X_train.values, label=y_train.values)

X_test = test_data_final.drop(['PassengerId','Has_Companions'],axis = 1)

dtest = xgb.DMatrix(X_test.values)

# モデルの訓練
model = xgb.train(params, dtrain)

# テストデータに対する予測（確率）
y_pred_proba = model.predict(dtest)

# 確率を 0 または 1 のクラスラベルに変換
y_pred = np.where(y_pred_proba > 0.5, 1, 0)


In [ ]:
# Create a DataFrame with the PassengerId and the predictions for submission
submission_xgb_1 = pd.DataFrame({
    "PassengerId": test_data_final['PassengerId'],
    "Perished": y_pred
})

# Show the first few entries of the submission file
display(submission_xgb_1.head(10))

submission_xgb_1.to_csv('/content/drive/MyDrive/GCI_compe_1/submission/submission_xgb_1.csv')

,PassengerId,Perished
0,892,1
1,893,0
2,894,1
3,895,1
4,896,0
5,897,1
6,898,0
7,899,1
8,900,0
9,901,1
